In [5]:
'''
参考文章 QONN in pennylanes
'''
import numpy as np
import pennylane as qml
from pennylane.operation import Operation

create a custom operator by class Operation

In [6]:
class RBS(Operation):
    num_params = 1
    num_wires = 2
    par_domain = "R"

    def __init__(self, theta, wires):
        self.theta = theta
        super().__init__(theta, wires=wires)

    def expand(self):
    # expand 定义了此量子操作的展开形式
        qml.Hadamard(wires=self.wires[0])
        qml.Hadamard(wires=self.wires[1])
        qml.CZ(wires=[self.wires[0],self.wires[1]])
        qml.RY(self.theta/2, wires=self.wires[0])
        qml.RY(-self.theta/2, wires=self.wires[1])
        qml.CZ(wires=[self.wires[0],self.wires[1]])
        qml.Hadamard(wires=self.wires[0])
        qml.Hadamard(wires=self.wires[1])

In [7]:
# dev = qml.device('default.qubit', wires=2)
# @qml.qnode(dev)
# def circuit():
#     qml.QubitStateVector([0,1,0,0]
#                         ,wires=[0,1])
#     RBS(np.pi, [0,1])
#     return qml.state()
# circuit()

# 使用矩阵直接生成RBS

In [8]:
# plan B,使用方程生成操作
def my_RBS(params, on_wire):
    """定义一个自定义的参数化二比特门"""
    qml.QubitUnitary(np.array([[1, 0, 0, 0],
                               [0, np.cos(params), np.sin(params), 0],
                               [0, -np.sin(params), np.cos(params), 0],
                               [0, 0, 0, 1]]),
                     wires=on_wire)


In [26]:
dev = qml.device('default.qubit', wires=4)
@qml.qnode(dev, diff_method="parameter-shift")
def circuit(params):
    qml.QubitStateVector(in_state
                        ,wires=[0,1,2,3])
    my_RBS(params[0], [0,1])
    my_RBS(params[1], [1,2])
    my_RBS(params[2], [0,1])
    my_RBS(params[3], [2,3])
    my_RBS(params[4], [1,2])
    return qml.expval(qml.PauliZ(0))

theta = np.array([0.1, 0.2, 0.3, 1, 1])
in_state = np.zeros(2**4)
in_state[1] = 1.0
print(circuit(theta))
print(qml.draw(circuit)(theta))
# 测试 grad函数

grad = qml.gradients.param_shift(circuit)(theta)
print(grad)

1.0
0: ─╭QubitStateVector(M0)─╭U(M1)────────╭U(M4)────────┤  <Z>
1: ─├QubitStateVector(M0)─╰U(M1)─╭U(M2)─╰U(M4)─╭U(M3)─┤     
2: ─├QubitStateVector(M0)────────╰U(M2)─╭U(M3)─╰U(M3)─┤     
3: ─╰QubitStateVector(M0)───────────────╰U(M3)────────┤     
()


数据读取

In [27]:
from openpyxl import load_workbook
# 1.打开 Excel 表格并获取表格名称
workbook = load_workbook(filename="Python招聘数据（全）.xlsx")
print(workbook.sheetnames)
# 2.通过 sheet 名称获取表格
sheet = workbook["Sheet1"]
print(sheet)
# 3.获取表格的尺寸大小(几行几列数据) 这里所说的尺寸大小，指的是 excel 表格中的数据有几行几列，针对的是不同的 sheet 而言。
print(sheet.dimensions)
# 4.获取表格内某个格子的数据
# 1 sheet["A1"]方式
cell1 = sheet["A1"]
cell2 = sheet["C11"]
print(cell1.value, cell2.value)
"""
workbook.active 打开激活的表格; sheet["A1"] 获取 A1 格子的数据; cell.value 获取格子中的值;
"""
# 4.2sheet.cell(row=, column=)方式
cell1 = sheet.cell(row = 1,column = 1)
cell2 = sheet.cell(row = 11,column = 3)
print(cell1.value, cell2.value)

# 5. 获取一系列格子
# 获取 A1:C2 区域的值
cell = sheet["A1:C2"]
print(cell)
for i in cell:
   for j in i:
       print(j.value)

FileNotFoundError: [Errno 2] No such file or directory: 'Python招聘数据（全）.xlsx'